In [208]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC

In [209]:
# Change paths as necessary

path_to_annotations = "trainset_gt_annotations.txt"
path_to_labels = "concepts_2011.txt"
path_to_data = "imagecleffeats/imageclef2011_feats/"

In [210]:
class image_processing:
    def __init__(self,annotations,all_labels,data):
        self.annotation = annotations
        self.labels = all_labels
        self.data = data
        
    #We only want images with labels from 9-12
    def filter_data(self):
        images = []
        labels = []
        with open(self.annotation,'r') as f:
            for line in f:
                label_arr = line.rstrip().split()
                if "1" in label_arr[10:14]:              
                    images.append(label_arr[0])
                    labels.append(np.array([int(i) for i in label_arr[10:14]]))
        return images,np.asarray(labels)
    
    def split_data(self,files,labels,datapath):
        yvals = labels
        xvals = np.array([np.load(datapath+file+"_ft.npy") for file in files])
        x_train,x_test,y_train,y_test = train_test_split(xvals,yvals,test_size=0.4,stratify=yvals,random_state=10)
        x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,test_size=1-0.625,stratify=y_test,random_state=10)
        return x_train,x_test,y_train,y_test,x_val,y_val
        

In [214]:
class MultiClassSVM:
    def __init__(self):
        self.models = []
    
    def data_processing(self, img_class, data):
        indexed = np.argmax(data,axis=-1)
        processed_data = np.zeros(indexed.shape)
        for i in range(len(indexed)):
                processed_data[i] = 1 if indexed[i] == img_class else 0
        return processed_data
    
    def save(self,x,y):
        data_x = ['xtrain','xtest','xval']
        data_y = ['ytrain','ytest','yval']
        for idx,datax in enumerate(x):
            np.save(data_x[idx],datax)
        for idx,datay in enumerate(y):
            np.save(data_y[idx],datay)
            
    
    def train(self,reg_param,xtrain, ytrain):
        self.models = [SVC(C=reg_param,kernel='linear') for i in range(4)]
        for i in range(4):
            processed_data = self.data_processing(i,ytrain)
            self.models[i].fit(xtrain, processed_data)
    
    def classwise_test(self,xval):
        decision_matrix = np.zeros((len(xval),4))
        for i in range(len(self.models)):
            decision_func = self.models[i].decision_function(xval)
            decision_matrix[:,i] = decision_func
        predicted_labels = np.argmax(decision_matrix, axis=-1)
        return predicted_labels
    
    def vanilla_acc(self,ytest,predicted):
        #compare predicted values and actual values. Average over all values
        assert len(ytest) == len(predicted),print("Lengths of gold and predicted are different",ytest.shape,predicted.shape)
        return np.sum(np.equal(ytest,predicted))/len(ytest)
    
    def classwise_acc(self, gold, predicted):
        #vanilla
        scores = []
        #how many predicted correspond to the class?
        for i in range(4):
            val_check = (gold == i)
            intermed_scores = self.vanilla_acc(gold[val_check],predicted[val_check]) #performs boolean masking
            scores.append(intermed_scores) 
        return np.mean(scores)
    
    def train_plus_val(self,reg,xtrain,xval,ytrain,yval,xtest,ytest):
        x_train = np.concatenate((xtrain,xval),axis=0)
        y_train = np.concatenate((ytrain,yval),axis=0)
        self.train(reg,x_train,y_train)
        y_val = np.argmax(yval,axis=-1)
        val_labels = self.classwise_test(xval)
        accuracy1 = self.classwise_acc(y_val,val_labels)

        true_values = np.argmax(ytest,axis=-1)
        test_labels = self.classwise_test(xtest)
        accuracy2 = self.classwise_acc(true_values,test_labels)
        return accuracy1,accuracy2

In [215]:
train = image_processing(path_to_annotations,path_to_labels,path_to_data)
images,labels = train.filter_data()
x_train,x_test,y_train,y_test,x_val,y_val = train.split_data(images,labels,path_to_data)
print("Training features:",x_train.shape,"Test features:",x_test.shape,
      "Training labels:",y_train.shape,"Test labels:",y_test.shape,"Validation features:",x_val.shape,"Validation Labels:",y_val.shape)

data_x = [x_train,x_test,x_val]
data_y = [y_train,y_test,y_val]


def one_vs_rest(x_train,x_test,y_train,y_test,x_val,y_val):
    reg_param = [0.001,0.01,0.1,0.1**0.5,1,10**0.5, 10, 1000**0.5]
    reg_scores = []
    true_labels = np.argmax(y_val,axis=-1)
    for reg in reg_param:
        mult_class = MultiClassSVM()
        mult_class.train(reg,x_train,y_train)
        learned = mult_class.classwise_test(x_val)
        accuracy = mult_class.classwise_acc(true_labels,learned)
        reg_scores.append(accuracy)
        print("C =", reg, ", Acc =",accuracy)

    best_c = max(reg_scores)
    c_value = reg_param[reg_scores.index(best_c)]
    print("Best C accuracy:",best_c,"Best C value:",c_value)
    return c_value

c_value = one_vs_rest(x_train,x_test,y_train,y_test,x_val,y_val)
new_mult_class = MultiClassSVM()
# new_mult_class.save(data_x,data_y)
val,test = new_mult_class.train_plus_val(c_value,x_train,x_val,y_train,y_val,x_test,y_test)
print("Train + validation set accuracy on val:",val)
print("Train + validation set accuracy on test:",test)


Training features: (811, 1024) Test features: (338, 1024) Training labels: (811, 4) Test labels: (338, 4) Validation features: (204, 1024) Validation Labels: (204, 4)
C = 0.001 , Acc = 0.3681757760984698
C = 0.01 , Acc = 0.5245147422076155
C = 0.1 , Acc = 0.4988176011596994
C = 0.31622776601683794 , Acc = 0.4814247268217119
C = 1 , Acc = 0.48329039846350297
C = 3.1622776601683795 , Acc = 0.48329039846350297
C = 10 , Acc = 0.48329039846350297
C = 31.622776601683793 , Acc = 0.48329039846350297
Best C accuracy: 0.5245147422076155 Best C value: 0.01
Train + validation set accuracy on val: 0.824831945636474
Train + validation set accuracy on test: 0.5096381806908122
